In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import sys, os, shutil
import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime
import xarray as xr

In [2]:
levelArray=['0', '1a', '1b', '1c', '2a', '2b', '2c', '3']
startId=1
endId=5

aw_apriori_baseDir = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/model/06282000'
aw_sim_basedir = os.path.join(aw_apriori_baseDir,'source_info/apriori_results')

valid_dir = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/valid/06282000'
domain_basename='DDS'
expers = range(startId,endId+1)

route_outFilePrefix = 'sflow'

# obs_file = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/model/obs/obs_flow.06282000.cfs.csv'
obs_file = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/model/obs/obs_flow.BBR_IN.cfs.csv'
obsFile = obs_file
obs_unit = 'cfs'
q_seg_index=1

output_baseDir = os.path.join(calib_dir,'analysis','9_plot_hydrograph')
if not os.path.exists(output_baseDir):
    os.makedirs(output_baseDir)

plot_output_dir = os.path.join(output_baseDir,'hydrographs')
if not os.path.exists(plot_output_dir):
    os.makedirs(plot_output_dir)


In [ ]:
events = ['10/30/16', '10/08/13','08/26/04','01/02/1997', '09/11/1991',
          '04/22/2010','04/29/2009', '04/15/2002',' 04/29/1999', '04/11/1996']


In [3]:
for complexity_level in levelArray:
    
    best_objs = []
    best_runs = []
    total_runs = []
    
    for experId in expers:
    
        domain_name=complexity_level+'_'+domain_basename+str(experId)
        ostModelFile = os.path.join(calib_dir,domain_name,'calib/OstModel0.txt')

        # Read OstModel0.txt 
        data = np.loadtxt(ostModelFile,skiprows=1)
        runs = data[:,0]
        objs = data[:,1]

        # Identify the best obj per calib experiment. 
        best_objs.append(np.min(objs))
        best_runs.append(runs[np.argmin(objs)])
        total_runs.append(len(runs))
    
    best_obj = np.nanmin(best_objs)
    best_obj_indx = np.nanargmin(best_objs)
    best_run = best_runs[best_obj_indx]
    best_exper = expers[best_obj_indx]
    total_run = total_runs[best_obj_indx]
    print('%s: best obj = %.6f, calib exper %d, best_run %d, total_run %d.'\
          %(complexity_level,best_obj,best_exper,best_run,total_run))   

    # --- save --- 
    domain_name=complexity_level+'_'+domain_basename+str(best_exper)
    src_dir = os.path.join(calib_dir,domain_name,'calib/output_archive/experiment1/run%d'%(best_run))
    dst_dir = os.path.join(output_baseDir,complexity_level+'_'+domain_basename)
    if os.path.exists(dst_dir):
        shutil.rmtree(dst_dir)
    shutil.copytree(src_dir, dst_dir)
    
    ofile = os.path.join(dst_dir,'best_run_source.txt')
    f = open(ofile, "w") 
    f.write('%s: best obj = %.6f, calib exper %d, best_run %d, total_run %d.'\
            %(complexity_level,best_obj,best_exper,best_run, total_run))
    f.close()
    

0: best obj = -0.927500, calib exper 1, best_run 135, total_run 979.
1a: best obj = -0.918689, calib exper 5, best_run 149, total_run 617.
1b: best obj = -0.936477, calib exper 2, best_run 126, total_run 541.
1c: best obj = -0.884804, calib exper 5, best_run 108, total_run 694.
2a: best obj = -0.925989, calib exper 5, best_run 719, total_run 1741.
2b: best obj = -0.893748, calib exper 4, best_run 70, total_run 357.
2c: best obj = -0.893670, calib exper 3, best_run 36, total_run 614.
3: best obj = -0.902371, calib exper 4, best_run 81, total_run 452.


#### Plot event hydrographs

In [4]:
for event in events:
    
    eventStartDate = datetime.strftime(datetime.strptime(event, '%m/%d/%Y') + datetime.timedelta(days=15), '%Y-%m-%d')     
    eventEndDate = datetime.strftime(datetime.strptime(event, '%m/%d/%Y') - datetime.timedelta(days=15), '%Y-%m-%d')         
    
    plot_output_dir = = os.path.join(output_baseDir,event)
    if not os.path.exists(plot_output_dir):
        os.makedirs(plot_output_dir)
    
    # Read sim and obs data
    for complexity_level in levelArray:

        print('---%s---'%(complexity_level))
        simVarName = 'IRFroutedRunoff'
        simFile = os.path.join(valid_dir,complexity_level+'_'+domain_basename, 
                               route_outFilePrefix+'.mizuRoute.nc')

        f    = xr.open_dataset(simFile)
        time = f['time'].values
        sim  = f[simVarName][:,(q_seg_index-1)].values #(time, segments)
        df_sim = pd.DataFrame({'sim':sim},index = time)
        df_sim.index = pd.to_datetime(df_sim.index)

        # --- read observed flow (cfs or cms) --- 
        df_obs = pd.read_csv(obsFile, index_col='Date', na_values=["-99.0","-999.0","-9999.0"],
                             parse_dates=True, infer_datetime_format=True)  
        df_obs.columns = ['obs']

        # convert obs from cfs to cms
        if obs_unit == 'cfs':
            df_obs = df_obs/35.3147    

        # --- # combine daily sim & obs timeseries df based on time index--- 
        df_sim_eval = df_sim.truncate(before=eventStartDate, after=eventEndDate)
        df_obs_eval = df_obs.truncate(before=eventStartDate, after=eventEndDate)
        df_merge = pd.concat([df_obs_eval, df_sim_eval], axis=1)
        df_merge = df_merge.dropna()
        df_merge.index = pd.to_datetime(df_merge.index)

    # --- Plot: part 1 --- 
    nrow, ncol = 2, 2
    lwd = 1.0
    
    fig, ax = plt.subplots(nrow, ncol,figsize=[7.08, 7.08*0.3*nrow], constrained_layout=True)
    fig.suptitle('Event '+event +' (complexity levels 0-1c)', fontsize='medium',weight='semibold')

    for i in range(nrow): 
        for j in range(ncol):

            comp_id = i*ncol + j
            complexity_level = levelArray[comp_id]  
            df_merge.plot(ax=ax[comp_id], color=['red','black'], linewidth=lwd)

        ax[i,j].set_xlabel('Date')
        ax[i,j].set_ylabel('Flow (cms)')

        subtitle = complexity_level
        ax[i,j].set_title(subtitle,fontsize='small',weight='semibold')
        ax[i,j].tick_params(axis='both', direction='out', labelsize='small')

    ofile = event+'_hydrograph_part1.png'
    plt.savefig(os.path.join(plot_output_dir,ofile), dpi=150)
    plt.close(fig) 
    
    # --- Plot: part 2 --- 
    nrow, ncol = 2, 2
    lwd = 1.0
    
    fig, ax = plt.subplots(nrow, ncol,figsize=[7.08, 7.08*0.3*nrow], constrained_layout=True)
    fig.suptitle('Event '+event +' (complexity levels 0-1c)', fontsize='medium',weight='semibold')

    for i in range(nrow): 
        for j in range(ncol):

            comp_id = 4 + i*ncol + j
            complexity_level = levelArray[comp_id]  
            df_merge.plot(ax=ax[comp_id], color=['red','black'], linewidth=lwd)

        ax[i,j].set_xlabel('Date')
        ax[i,j].set_ylabel('Flow (cms)')

        subtitle = complexity_level
        ax[i,j].set_title(subtitle,fontsize='small',weight='semibold')
        ax[i,j].tick_params(axis='both', direction='out', labelsize='small')

    ofile = event+'_hydrograph_part2.png'
    plt.savefig(os.path.join(plot_output_dir,ofile), dpi=150)
    plt.close(fig)  

---0---
correlations (AJ, WY):  0.7311148751974688 0.7484421119251399
plot layout
plot monthly
plot daily calibration period
plot monthly long term averages
plot scatter for water year mean flow
plot scatter for for spring runoff period
other plot details
save plot
---1a---
correlations (AJ, WY):  0.6820894559752355 0.7015571683076651
plot layout
plot monthly
plot daily calibration period
plot monthly long term averages
plot scatter for water year mean flow
plot scatter for for spring runoff period
other plot details
save plot
---1b---
correlations (AJ, WY):  0.7511194054681285 0.726461543114851
plot layout
plot monthly
plot daily calibration period
plot monthly long term averages
plot scatter for water year mean flow
plot scatter for for spring runoff period
other plot details
save plot
---1c---
correlations (AJ, WY):  0.593809360068252 0.6731032961873309
plot layout
plot monthly
plot daily calibration period
plot monthly long term averages
plot scatter for water year mean flow
plot s

In [ ]:

        comp_id = i*ncol + j
        complexity_level = levelArray[comp_id]  
        ax[i,j].plot(df_merge.index, df_merge['sim'], 'red', label='sim', linewidth=lwd)
        ax[i,j].plot(df_merge.index, df_merge['obs'], 'black', label='obs', linewidth=lwd)

        ax[i,j].set_xlabel('Date',fontsize='small')
        ax[i,j].set_ylabel('Flow (cms)',fontsize='small')

        date_form = DateFormatter("%Y-%m-%d")
        ax[i,j].xaxis.set_major_formatter(date_form)
#             ax[i,j].set_xticklabels(rotation = 'vertical')

        subtitle = complexity_level
        ax[i,j].set_title(subtitle,fontsize='small',weight='semibold')
        ax[i,j].tick_params(axis='both', direction='out', labelsize='small')

        ax[i,j].legend(fontsize='small')